In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Oct_dict_JSON.json', 'r') as json_file:
    link_day_minute_Oct_dict_JSON = json.load(json_file)

In [10]:
week_day_Oct_list = [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30, 31]
#week_day_Oct_list = [2, 9, 16, 23, 30]

In [11]:
link_day_minute_Oct_list = []
for link_idx in range(24):
    for day in week_day_Oct_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Oct_list.append(link_day_minute_Oct_dict_JSON[key] ['MD_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Oct_list)

66240

In [13]:
x = np.matrix(link_day_minute_Oct_list)
x = np.matrix.reshape(x, 24, 2760)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2760)

In [15]:
x[:,:2]

matrix([[  2727.82029999,   3098.14707426],
        [  3905.27489626,   3383.09522346],
        [  4076.9221011 ,   4954.11429807],
        [  5444.09755492,   3967.35332247],
        [  8151.04287328,   7726.21196495],
        [  3232.76407051,   5407.99509162],
        [  4306.99504063,   3958.38701888],
        [  4445.98004865,   6226.92933848],
        [  7036.15464385,   5669.05449651],
        [  6183.89258262,   7441.75454096],
        [  7099.80125721,   7438.15549305],
        [  6893.63379938,   7473.50063928],
        [  6760.33843474,   4807.59484749],
        [  4335.54356474,   3902.29331208],
        [  3971.1852384 ,   6541.69569595],
        [  4669.25592226,   2396.27762553],
        [  4696.30435731,   4970.38202738],
        [  3878.06842245,   5096.40991128],
        [ 12994.5244703 ,   6508.94258314],
        [  5317.60855205,   4374.21012242],
        [  3259.84683916,   3849.32031686],
        [  6093.22968301,   6393.75807734],
        [  1827.27125926,   2835

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [3e-02, 8e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.73312431e+06  0.00000000e+00  4.21e+03 1.57e+02  9.55e+05     0s
   1   9.05376881e+05 -1.79259484e+06  9.08e+02 2.02e+02  2.49e+05     0s
   2   8.91848730e+05 -1.37925563e+06  5.91e+01 1.33e+01  5.09e+04     0s
   3   8.01339318e+04 -3.24795500e+05  5.91e-05 5.68e-14  7.2

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Oct_weekday_MD.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[9.462975309492649e-11,
 5449.364071826014,
 1.038718039546307e-10,
 9.02809734917734e-07,
 1242.6407235392342,
 1.513585729894109e-06,
 3.071151733491655e-08,
 0.0009159566254446638,
 334.7691687778002,
 3222.6548462807077,
 9.02808803872969e-07,
 246.46059243451703,
 1.5135831133171777e-06,
 1.5222504751630477e-06,
 1955.231625911226,
 974.9961225114952,
 9.028095946367451e-07,
 9.028097893579916e-07,
 1490.743811039035,
 10357.363890126006,
 1.513263438906754e-06,
 2154.4646684687473,
 1984.9132087159398,
 363.26589305022395,
 1211.477257833448,
 991.9062031791195,
 587.8380184491323,
 3594.8575191766704,
 0.1956839614874326,
 0.14882986411636878,
 587.6782299939811,
 1870.2090397718173,
 5259.939961504766,
 929.2309091829682,
 547.2548358673282,
 3178.97578597615,
 1355.2397692570325,
 3.401693690071889e-05,
 1135.4112108828474,
 2567.1073721394496,
 719.6477354954365,
 490.9097732275619,
 3.366280767832955e-07,
 3.3662804852294245e-07,
 7888.050577499424,
 1.3659252667306835e-07,
